# 神经网络的学习

In [11]:
import os, sys
import numpy as np
sys.path.append(os.pardir)

import common.functions as functions

In [ ]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
functions.mean_squared_error(np.array(y), np.array(t))

1.5